In [1]:
from datetime import date, datetime

import pandas as pd
from pandasql import sqldf
import plotly.express as px
from sklearn.cluster import KMeans
from pandas_profiling import ProfileReport

pysqldf = lambda q: sqldf(q, globals())

In [2]:
features = pd.read_csv('data_extractions/songs_features.csv')
played =  pd.read_csv('data_extractions/played_tracks.csv')
ids = pd.read_csv('data_extractions/tracks_ids.csv')

#hard fixing the column names for the played file
played.rename(columns={'0':'artist','1':'song_name','2':'unix_timestamp'}, inplace=True)

In [3]:
q = """
SELECT p.artist, p.song_name, p.unix_timestamp, f.*
FROM played as p
LEFT JOIN ids as i on i.band_name = p.artist and i.song_name = p.song_name
LEFT JOIN features as f on f.id = i.sp_id
"""
raw = pysqldf(q)

#### Organize the dataset

In [ ]:
#take a look at the NaNs first to be sure 
raw[raw['id'].isna() == False][['artist', 'song_name']].nunique()
#there are 1051 unique songs we couldn't find the id (17.4%)
##I'll find a way to deal with this another time

In [4]:
#unix timestamp to readable datetime
raw['play_timestamp'] = pd.to_datetime(raw['unix_timestamp'], unit='s')
#remove the tracks we could not find the spotify id
raw.dropna(inplace= True )
#drop some columns we won't need (or that I don't want to use as a feature)
raw.drop(columns = ['unix_timestamp', 'key', 'mode', 'speechiness', 'liveness', 'type', 'uri', 'track_href', 'analysis_url'], axis = 1, inplace  =True)

Pandas profiling all the song features
- just remove the `#`s to not overload the notebook

In [ ]:
#profile = ProfileReport(
#    raw[['danceability', 'energy', 'loudness','acousticness', 'instrumentalness', 'valence', 'tempo', 'duration_ms','time_signature']], 
#    title='Song features 2019',
#    html={'style':{'full_width':True}})
#profile.to_notebook_iframe()

## K-means

In [ ]:
raw.groupby('id').count().describe()['play_timestamp']

In [5]:
# Cleaning up a bit more so we can generate playlists only with songs oftenr listened to
##I'd like to check the results considering only the songs that I've listened to more than once
q = """
with base as(
    SELECT DISTINCT id,
        danceability,
        energy,
        loudness,
        acousticness,
        instrumentalness,
        valence,
        tempo,
        duration_ms,
        time_signature, 
        COUNT(*) as count
    FROM raw
    GROUP BY 1,2,3,4,5,6,7,8,9,10)
SELECT * FROM base WHERE count > 1
"""
kdf = pysqldf(q)

In [6]:
fig = px.scatter_matrix(kdf.drop("id",axis=1),
width=1200, height=1600)
fig.show()